##문제
제공된 학습용 데이터(train.csv)는 서울시 각 시군구별 건물 및 시설 현황 데이터이다.
학습용 데이터를 활용하여 총가스사용량을 예측하는 모형을 개발하고, 모형을 평가용 데이터(test.csv)에 적용하여 총가스사용량을 예측하시오.

예측 결과는 아래의 [유의 사항]과 [제출 형식]을 준수하여, CSV 파일(result.csv)을 생성하는 코드를 제출하시오.

---

[유의 사항]

① 예측 결과는 RMSE(Root Mean Squared Error) 평가 지표에 따라 평가함

② 타겟 변수(총가스사용량)는 일부 값이 0으로 기재되어 있으며, 이는 결측치를 대체한 값임.

[제출 형식]

① CSV 파일명: result.csv

② 예측 총가스사용량 컬럼명: pred

③ 제출 컬럼 개수: pred 컬럼 1개

④ 평가용 데이터 개수와 예측 결과 데이터 개수 일치: 1,476개

---

[제공 데이터]
- train.csv: 학습용 데이터, 약 3,196개
- test.csv: 평가용 데이터, 약 1,476개
평가용 데이터에는 총가스사용량 컬럼 미제공

[CSV 파일 형식 및 확인 방법]
- CSV 파일명: result.csv
- 예측 총가스사용량 컬럼명: pred
- 제출 CSV 파일 예시
~~~
pred
8790
14748
21560
761
~~~

In [27]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_test.csv")

In [28]:
# train.head()
target = train.pop('총가스사용량')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_train, y_train)
rf_pred_val = rf.predict(X_val)

import lightgbm as lgb
lgbm = lgb.LGBMRegressor(random_state=0)
lgbm.fit(X_train, y_train)
lgbm_pred_val = lgbm.predict(X_val)

from sklearn.metrics import root_mean_squared_error
rf_rmse = root_mean_squared_error(rf_pred_val, y_val)
print('rf: ', rf_rmse)
lgbm_rmse = root_mean_squared_error(lgbm_pred_val, y_val)
print('lgbm: ', lgbm_rmse)

pred = rf.predict(test)
result = pd.DataFrame({'pred': pred})
result.to_csv('result.csv', index=False)

pd.read_csv('result.csv')

rf:  960.485846380754
lgbm:  1064.8095758723994


,pred
0,9787.863633
1,10770.220091
2,9137.436055
3,9787.863633
4,11234.287608
...,...
1471,12258.710856
1472,10904.053927
1473,9895.018350
1474,9827.441850


## EDA

In [3]:

# 문제정의
# 평가: RMSE
# target: 총가스사용량
# 최종파일: result.csv(컬럼 1개 pred)

# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_test.csv")

# 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

print("\n ===== 데이터 샘플 =====")
print(train.head())

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train object컬럼 unique 수 =====")
print(train.describe(include='object'))

print("\n ===== test object컬럼 unique 수 =====")
print(test.describe(include='object'))

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum().sum())

print("\n ===== target 기술 통계 =====")
print(train['총가스사용량'].describe())


===== 데이터 크기 =====
Train Shape: (3196, 6)
Test Shape: (1476, 5)

 ===== 데이터 샘플 =====
  시군구명  생활및판매  공공문화  복지의료  업무오락체육   총가스사용량
0  구로구      2     0     0       0   9077.8
1  구로구      6     0     1       2  10105.5
2  구로구     27     0     0       0   8603.6
3  구로구      2     0     0       0  11076.8
4  구로구     83     0     1      19  10781.4

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3196 entries, 0 to 3195
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시군구명    3196 non-null   object 
 1   생활및판매   3196 non-null   int64  
 2   공공문화    3196 non-null   int64  
 3   복지의료    3196 non-null   int64  
 4   업무오락체육  3196 non-null   int64  
 5   총가스사용량  3196 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 149.9+ KB
None

 ===== train object컬럼 unique 수 =====
        시군구명
count   3196
unique    21
top      강남구
freq     715

 ===== test object컬럼 unique 수 =====
        시군구명
count  

In [4]:
# 데이터
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_test.csv")

# 데이터 전처리
target = train.pop('총가스사용량')

# 원핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)


# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

# RMSE
from sklearn.metrics import root_mean_squared_error
print(root_mean_squared_error(y_val, pred))

# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print(root_mean_squared_error(y_val, pred))

# 최종 제출 파일
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv("result.csv", index=False)

(3196, 5) (1476, 5)
(3196, 25) (1476, 25)
960.485846380754
1064.8095758723994


### 심화(0 처리)

In [5]:
# 데이터
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/10_2/gas_test.csv")

# 데이터 전처리
cond = train['총가스사용량'] == 0
train = train[~cond]
target = train.pop('총가스사용량')

# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

# RMSE
from sklearn.metrics import root_mean_squared_error
print(root_mean_squared_error(y_val, pred))

# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print(root_mean_squared_error(y_val, pred))

# 최종 제출 파일
pred = rf.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv("result.csv", index=False)

673.8026644431536
741.9117749289708
